In [ ]:
import gradio as gr
from transformers import pipeline
import pandas as pd
import altair as alt

zer = pipeline("zero-shot-classification")

def analyze_text(text, labels):

    if not labels:  
        labels = ['Chinese', 'Indian', 'Italian', 
                  'Mexican', 'Southern Usa', 
                  'Cajun Creole', 'French', 'Thai']
    data = zer(text, labels)
    data = pd.DataFrame(data)
    data['scores'] = data['scores'].round(2)
    data = data.rename(columns={"sequence": "Recipe", "scores": "Score", 
                                "labels": "Cuisine"})
    data_2 = data.drop(columns=["Recipe"], axis=1)

    color_scale = alt.Scale(range=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f'])

    plot = alt.Chart(data_2).mark_bar(
        opacity=0.8,
        size= 30
    ).encode(
        x=alt.X('Cuisine', title='Cuisine', sort=alt.EncodingSortField(field='Score',order='descending'), axis=alt.Axis(labelAngle=0, titleAnchor='start')),
        y=alt.Y('Score', title='Score', axis=alt.Axis(titleAnchor='start')),
        color=alt.Color('Cuisine', scale=color_scale),
        tooltip=[alt.Tooltip('Score',title='Matchscore of the Cuisine')]
    ).properties(
    width=600,
    height=400,
    title='Matchscore of the Cuisines'
    ).configure_axis(
    grid=False
    ).configure_view(
    strokeWidth=0
    )



    return data_2, plot

checkbox = gr.CheckboxGroup(['Chinese','Indian','Italian',
                             'Mexican','Southern Usa', 'Cajun Creole', 
                             'French', 'thai'], label="Choose countries", info="If nothing is selected, all options will be selected automatically.")

demo = gr.Interface(
    fn=analyze_text,
    inputs=["text", checkbox],
    outputs=["dataframe","plot"],
    title=" Zero-Shot Recipe Classification"
)

demo.launch()